In [1]:
import minsearch

In [2]:
import json

In [3]:
import os
os.chdir("/home/kantundpeterpan/projects/zoomcamp/zcllm/week1")

In [4]:
with open("./documents.json", "rt") as f:
    docs_raw = json.load(f)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields = ['course'] # for exact filtering
)

In [7]:
q = "the course has already started, can I still enroll?"

In [8]:
index.fit(documents)

In [9]:
boost = {
    'question': 3.0,
    'section': 0.5
}

res = index.search(
    query = q,
    filter_dict = {
        'course':'data-engineering-zoomcamp'
    },
    boost_dict = boost,
    num_results = 5
)

In [10]:
res

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# Generation: Provide the retrieved documents to the LLM 

In [11]:
from openai import OpenAI

In [12]:
with open('/home/kantundpeterpan/projects/llm_api_keys/OPENAI_API_KEY', 'r') as f:
    client = OpenAI(api_key=f.read().strip())

In [13]:
client.chat.completions.create(
    model = 'gpt-4o',
    messages = [
        {'role':"user", "content":q}
    ]
)

ChatCompletion(id='chatcmpl-BeSt9xC1xKwD2OyPqBOsMhbACtVrr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The ability to enroll in a course after it has started can depend on several factors such as the institution's policies, the type of course, and availability of space. Here are a few steps you can take:\n\n1. **Contact the Instructor or Department**: Reach out to the instructor or department offering the course to inquire about late enrollment policies. They might allow you to join if there’s still space and you can catch up on missed materials.\n\n2. **Check Institution Policies**: Review the institution's enrollment policies on their website or in the student handbook to understand the official stance on late enrollments.\n\n3. **Consider Administrative Approval**: Sometimes, late enrollment requires approval from a department chair or academic advisor, especially if it involves exceptions to standard policies.\n\n4. **Online

In [14]:
prompt_template = """
You are a course teaching assistant.

Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering
the QUESTION.
If the CONTEXT does not contain the answer, output NONE

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [15]:
context = ""

for doc in res:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [16]:
prompt = prompt_template.format(
    question=q, context = context
).strip()

In [17]:
print(client.chat.completions.create(
    model = 'gpt-4o',
    messages = [
        {'role':"user", "content":prompt}
    ]
).choices[0].message.content)

Yes, you can still enroll in the course even after it has started, but be aware that there will be deadlines for turning in the final projects.


In [18]:
def search(query):
    
    boost = {
        'question': 3.0,
        'section': 0.5
    }

    res = index.search(
        query = query,
        filter_dict = {
            'course':'data-engineering-zoomcamp'
        },
        boost_dict = boost,
        num_results = 5
    )
    
    return res

In [19]:
def build_prompt(query, search_results):
    
    prompt_template = """
    You are a course teaching assistant.

    Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering
    the QUESTION.
    If the CONTEXT does not contain the answer, output NONE

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()
    
    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(
        question=query, context = context
    ).strip()
    
    return prompt

In [20]:
def llm(prompt):
    return client.chat.completions.create(
        model = 'gpt-4o',
        messages = [
            {'role':"user", "content":prompt}
        ]
    ).choices[0].message.content

In [21]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [22]:
rag('can i enroll after the course started ?')

"Yes, you can still join the course after it has started. Be aware that there will be deadlines for turning in the final projects, so it's advisable not to leave everything for the last minute."

# Index the documents with ElasticSearch

In [23]:
from elasticsearch import Elasticsearch

In [24]:
es = Elasticsearch('http://localhost:9200')

In [25]:
es.info()

ObjectApiResponse({'name': '65ad251f2afd', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gJTfKtocQXuXjpgE_YvfSQ', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course_question'

es.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_question'})

In [27]:
for doc in documents:
    es.index(index = index_name, document=doc)

In [28]:
query = 'i just discovered the course, can i still join ?'

In [34]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index = index_name, body = search_query)
    
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
        
    return result_docs

In [35]:
elastic_search(query)

[]

In [31]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [32]:
rag(query)

'Yes, you can still join the course even after the start date. You are eligible to submit the homework, but be mindful of the deadlines for turning in the final projects.'